In [1]:
import os
from dotenv import load_dotenv

from datetime import datetime

from binance.client import Client

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

KeyboardInterrupt: 

In [2]:
load_dotenv()
API_KEY    = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

client = Client(API_KEY, API_SECRET)

In [3]:
symbol = 'BTCUSDT'
interval = '1h'
window = 60

start_time = int(datetime(2020,1,1,0,0).timestamp() * 1000)
end_time   = int(datetime(2021,12,31,0,0).timestamp() * 1000)

In [4]:
kline = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start_time, end_str=end_time)

In [5]:
columns = ['index', 'Close']
data = pd.DataFrame(kline)
data = data.iloc[:, [0, 4]]
data.columns  = columns
data['index'] = pd.to_datetime(data['index'], unit='ms')
data.set_index('index', inplace=True)
data = data.astype(float)
data.dropna(inplace=True)

In [6]:
data_train = data.iloc[:int(data.shape[0]*0.6), ]
data_test  = data.iloc[int(data.shape[0]*0.6):, ]

In [8]:
training_set = data_train.values

In [ ]:
training_set = data_train.values

scaler = MinMaxScaler(feature_range=(0,1))
scaled_training_set = scaler.fit_transform(training_set)

In [8]:
X_train = []
y_train = []
for i in range(window, scaled_training_set.shape[0]):
    X_train.append(scaled_training_set[i-window:i, 0])
    y_train.append(scaled_training_set[i, 0])
X_train = np.array(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.array(y_train)

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units=50,
                   return_sequences=True,
                   input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,
                   return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50,
                   return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

In [ ]:
regressor.compile(optimizer='adam',
                  loss='mean_squared_error')
regressor.fit(X_train, y_train,
              epochs=40,
              batch_size=32)

https://www.simplilearn.com/tutorials/machine-learning-tutorial/stock-price-prediction-using-machine-learning

In [ ]:
testing_set = data_test.values
scaled_testing_set = scaler.transform(testing_set)

X_test = []
y_test = []
for i in range(window, scaled_testing_set.shape[0]):
    X_test.append(scaled_testing_set[i-window:i, 0])
    y_test.append(testing_set[i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y_test = np.array(y_test)

In [ ]:
predicted = regressor.predict(X_test)
predicted = scaler.inverse_transform(predicted)

In [ ]:
plt.plot(y_test, color='red', label='Actual')
plt.plot(predicted, color='blue', label='Predicted')
plt.title('LSTM BTC 1h')
plt.legend() 